# Week 13: Preprocessing

Source: [MNE-Python](https://mne.tools/stable/index.html)

## 1. Overview of artifact detection

This tutorial covers the basics of artifact detection, and introduces the
artifact detection tools available in MNE-Python.

We begin as always by importing the necessary Python modules and loading some
`example data`:

In [ ]:
import os

import numpy as np

import mne

sample_data_folder = mne.datasets.sample.data_path()
sample_data_raw_file = os.path.join(
    sample_data_folder, "MEG", "sample", "sample_audvis_raw.fif"
)
raw = mne.io.read_raw_fif(sample_data_raw_file)
raw.crop(0, 60).load_data()  # just use a fraction of data for speed here

## What are artifacts?

Artifacts are parts of the recorded signal that arise from sources other than
the source of interest (i.e., neuronal activity in the brain). As such,
artifacts are a form of interference or noise relative to the signal of
interest. There are many possible causes of such interference, for example:

- Environmental artifacts
    - Persistent oscillations centered around the `AC power line frequency`_
      (typically 50 or 60 Hz)
    - Brief signal jumps due to building vibration (such as a door slamming)
    - Electromagnetic field noise from nearby elevators, cell phones, the
      geomagnetic field, etc.

- Instrumentation artifacts
    - Electromagnetic interference from stimulus presentation (such as EEG
      sensors picking up the field generated by unshielded headphones)
    - Continuous oscillations at specific frequencies used by head position
      indicator (HPI) coils
    - Random high-amplitude fluctuations (or alternatively, constant zero
      signal) in a single channel due to sensor malfunction (e.g., in surface
      electrodes, poor scalp contact)

- Biological artifacts
    - Periodic `QRS`_-like signal patterns (especially in magnetometer
      channels) due to electrical activity of the heart
    - Short step-like deflections (especially in frontal EEG channels) due to
      eye movements
    - Large transient deflections (especially in frontal EEG channels) due to
      blinking
    - Brief bursts of high frequency fluctuations across several channels due
      to the muscular activity during swallowing

There are also some cases where signals from within the brain can be
considered artifactual. For example, if a researcher is primarily interested
in the sensory response to a stimulus, but the experimental paradigm involves
a behavioral response (such as button press), the neural activity associated
with the planning and executing the button press could be considered an
artifact relative to signal of interest (i.e., the evoked sensory response).

<div class="alert alert-info"><h4>Note</h4><p>Artifacts of the same genesis may appear different in recordings made by
    different EEG or MEG systems, due to differences in sensor design (e.g.,
    passive vs. active EEG electrodes; axial vs. planar gradiometers, etc).</p></div>


## What to do about artifacts

There are 3 basic options when faced with artifacts in your recordings:

1. *Ignore* the artifact and carry on with analysis
2. *Exclude* the corrupted portion of the data and analyze the remaining data
3. *Repair* the artifact by suppressing artifactual part of the recording
   while (hopefully) leaving the signal of interest intact

There are many different approaches to repairing artifacts, and MNE-Python
includes a variety of tools for artifact repair, including digital filtering,
independent components analysis (ICA), Maxwell filtering / signal-space
separation (SSS), and signal-space projection (SSP). Separate tutorials
demonstrate each of these techniques for artifact repair. Many of the
artifact repair techniques work on both continuous (raw) data and on data
that has already been epoched (though not necessarily equally well); some can
be applied to `memory-mapped`_ data while others require the data to be
copied into RAM. Of course, before you can choose any of these strategies you
must first *detect* the artifacts, which is the topic of the next section.


## Artifact detection

MNE-Python includes a few tools for automated detection of certain artifacts
(such as heartbeats and blinks), but of course you can always visually
inspect your data to identify and annotate artifacts as well.

We saw in `the introductory tutorial <tut-overview>` that the example
data includes :term:`SSP projectors <projector>`, so before we look at
artifacts let's set aside the projectors in a separate variable and then
remove them from the :class:`~mne.io.Raw` object using the
:meth:`~mne.io.Raw.del_proj` method, so that we can inspect our data in it's
original, raw state:

In [ ]:
ssp_projectors = raw.info["projs"]
raw.del_proj()

### Low-frequency drifts

Low-frequency drifts are most readily detected by visual inspection using the
basic :meth:`~mne.io.Raw.plot` method, though it is helpful to plot a
relatively long time span and to disable channel-wise DC shift correction.
Here we plot 60 seconds and show all the magnetometer channels:

In [ ]:
mag_channels = mne.pick_types(raw.info, meg="mag")
raw.plot(duration=60, order=mag_channels, n_channels=len(mag_channels), remove_dc=False)

Low-frequency drifts are readily removed by high-pass filtering at a fairly
low cutoff frequency (the wavelength of the drifts seen above is probably
around 20 seconds, so in this case a cutoff of 0.1 Hz would probably suppress
most of the drift).

### Power line noise

Power line artifacts are easiest to see on plots of the spectrum, so we'll
use :meth:`~mne.io.Raw.compute_psd` to illustrate.

In [ ]:
fig = raw.compute_psd(tmax=np.inf, fmax=250).plot(
    average=True, amplitude=False, picks="data", exclude="bads"
)
# add some arrows at 60 Hz and its harmonics:
for ax in fig.axes[1:]:
    freqs = ax.lines[-1].get_xdata()
    psds = ax.lines[-1].get_ydata()
    for freq in (60, 120, 180, 240):
        idx = np.searchsorted(freqs, freq)
        ax.arrow(
            x=freqs[idx],
            y=psds[idx] + 18,
            dx=0,
            dy=-12,
            color="red",
            width=0.1,
            head_width=3,
            length_includes_head=True,
        )

Here we see narrow frequency peaks at 60, 120, 180, and 240 Hz — the power
line frequency of the USA (where the sample data was recorded) and its 2nd,
3rd, and 4th harmonics. Other peaks (around 25 to 30 Hz, and the second
harmonic of those) are probably related to the heartbeat, which is more
easily seen in the time domain using a dedicated heartbeat detection function
as described in the next section.

### Heartbeat artifacts (ECG)

MNE-Python includes a dedicated function
:func:`~mne.preprocessing.find_ecg_events` in the :mod:`mne.preprocessing`
submodule, for detecting heartbeat artifacts from either dedicated ECG
channels or from magnetometers (if no ECG channel is present). Additionally,
the function :func:`~mne.preprocessing.create_ecg_epochs` will call
:func:`~mne.preprocessing.find_ecg_events` under the hood, and use the
resulting events array to extract epochs centered around the detected
heartbeat artifacts. Here we create those epochs, then show an image plot of
the detected ECG artifacts along with the average ERF across artifacts. We'll
show all three channel types, even though EEG channels are less strongly
affected by heartbeat artifacts:

In [ ]:
ecg_epochs = mne.preprocessing.create_ecg_epochs(raw)
ecg_epochs.plot_image(combine="mean")

The horizontal streaks in the magnetometer image plot reflect the fact that
the heartbeat artifacts are superimposed on low-frequency drifts like the one
we saw in an earlier section; to avoid this you could pass
``baseline=(-0.5, -0.2)`` in the call to
:func:`~mne.preprocessing.create_ecg_epochs`.
You can also get a quick look at the
ECG-related field pattern across sensors by averaging the ECG epochs together
via the :meth:`~mne.Epochs.average` method, and then using the
:meth:`mne.Evoked.plot_topomap` method:

In [ ]:
avg_ecg_epochs = ecg_epochs.average().apply_baseline((-0.5, -0.2))

Here again we can visualize the spatial pattern of the associated field at
various times relative to the peak of the EOG response:

In [ ]:
avg_ecg_epochs.plot_topomap(times=np.linspace(-0.05, 0.05, 11))

Or, we can get an ERP/F plot with :meth:`~mne.Evoked.plot` or a combined
scalp field maps and ERP/F plot with :meth:`~mne.Evoked.plot_joint`. Here
we've specified the times for scalp field maps manually, but if not provided
they will be chosen automatically based on peaks in the signal:

In [ ]:
avg_ecg_epochs.plot_joint(times=[-0.25, -0.025, 0, 0.025, 0.25])

### Ocular artifacts (EOG)

Similar to the ECG detection and epoching methods described above, MNE-Python
also includes functions for detecting and extracting ocular artifacts:
:func:`~mne.preprocessing.find_eog_events` and
:func:`~mne.preprocessing.create_eog_epochs`. Once again we'll use the
higher-level convenience function that automatically finds the artifacts and
extracts them in to an :class:`~mne.Epochs` object in one step. Unlike the
heartbeat artifacts seen above, ocular artifacts are usually most prominent
in the EEG channels, but we'll still show all three channel types. We'll use
the ``baseline`` parameter this time too; note that there are many fewer
blinks than heartbeats, which makes the image plots appear somewhat blocky:

In [ ]:
eog_epochs = mne.preprocessing.create_eog_epochs(raw, baseline=(-0.5, -0.2))
eog_epochs.plot_image(combine="mean")
eog_epochs.average().plot_joint()

### 2. Handling bad channels

This tutorial covers manual marking of bad channels and reconstructing bad channels based on good signals at other sensors.

As usual we’ll start by importing the modules we need, and loading some example data:

In [ ]:
import os
from copy import deepcopy

import numpy as np

import mne

sample_data_folder = mne.datasets.sample.data_path()
sample_data_raw_file = os.path.join(
    sample_data_folder, "MEG", "sample", "sample_audvis_raw.fif"
)
raw = mne.io.read_raw_fif(sample_data_raw_file, verbose=False)

## Marking bad channels

Sometimes individual channels malfunction and provide data that is too noisy
to be usable. MNE-Python makes it easy to ignore those channels in the
analysis stream without actually deleting the data in those channels. It does
this by
keeping track of the bad channel indices in a list and looking at that list
when doing analysis or plotting tasks. The list of bad channels is stored in
the ``'bads'`` field of the :class:`~mne.Info` object that is attached to
:class:`~mne.io.Raw`, :class:`~mne.Epochs`, and :class:`~mne.Evoked` objects.

In [ ]:
print(raw.info["bads"])

Here you can see that the :file:`.fif` file we loaded from disk must have
been keeping track of channels marked as "bad" — which is good news, because
it means any changes we make to the list of bad channels will be preserved if
we save our data at intermediate stages and re-load it later. Since we saw
above that ``EEG 053`` is one of the bad channels, let's look at it alongside
some other EEG channels to see what's bad about it. We can do this using the
standard :meth:`~mne.io.Raw.plot` method, and instead of listing the channel
names one by one (``['EEG 050', 'EEG 051', ...]``) we'll use a `regular
expression`_ to pick all the EEG channels between 050 and 059 with the
:func:`~mne.pick_channels_regexp` function (the ``.`` is a wildcard
character):

In [ ]:
picks = mne.pick_channels_regexp(raw.ch_names, regexp="EEG 05.")
raw.plot(order=picks, n_channels=len(picks))

We can do the same thing for the bad MEG channel (``MEG 2443``). Since we
know that Neuromag systems (like the one used to record the example data) use
the last digit of the MEG channel number to indicate sensor type, here our
`regular expression`_ will pick all the channels that start with 2 and end
with 3:

In [ ]:
picks = mne.pick_channels_regexp(raw.ch_names, regexp="MEG 2..3")
raw.plot(order=picks, n_channels=len(picks))

Notice first of all that the channels marked as "bad" are plotted in a light
gray color in a layer behind the other channels, to make it easy to
distinguish them from "good" channels. The plots make it clear that ``EEG
053`` is not picking up scalp potentials at all, and ``MEG 2443`` looks like
it's got a lot more internal noise than its neighbors — its signal is a few
orders of magnitude greater than the other MEG channels, making it a clear
candidate for exclusion.

If you want to change which channels are marked as bad, you can edit
``raw.info['bads']`` directly; it's an ordinary Python :class:`list` so the
usual list methods will work:

In [ ]:
original_bads = deepcopy(raw.info["bads"])
raw.info["bads"].append("EEG 050")  # add a single channel
raw.info["bads"].extend(["EEG 051", "EEG 052"])  # add a list of channels
bad_chan = raw.info["bads"].pop(-1)  # remove the last entry in the list
raw.info["bads"] = original_bads  # change the whole list at once

Note: Blocking execution
```
    If you want to build an interactive bad-channel-marking step into an
    analysis script, be sure to include the parameter ``block=True`` in your
    call to ``raw.plot()`` or ``epochs.plot()``. This will pause the script
    while the plot is open, giving you time to mark bad channels before
    subsequent analysis or plotting steps are executed. This can be
    especially helpful if your script loops over multiple subjects.
```

You can also interactively toggle whether a channel is marked "bad" in the
plot windows of ``raw.plot()`` or ``epochs.plot()`` by clicking on the
channel name along the vertical axis (in ``raw.plot()`` windows you can also
do this by clicking the channel's trace in the plot area). The ``bads`` field
gets updated immediately each time you toggle a channel, and will retain its
modified state after the plot window is closed.

The list of bad channels in the :class:`mne.Info` object's ``bads`` field is
automatically taken into account in dozens of functions and methods across
the MNE-Python codebase. This is done consistently with a parameter
``exclude='bads'`` in the function or method signature. Typically this
``exclude`` parameter also accepts a list of channel names or indices, so if
you want to *include* the bad channels you can do so by passing
``exclude=[]`` (or some other list of channels to exclude). For example:

In [ ]:
# default is exclude='bads':
good_eeg = mne.pick_types(raw.info, meg=False, eeg=True)
all_eeg = mne.pick_types(raw.info, meg=False, eeg=True, exclude=[])
print(np.setdiff1d(all_eeg, good_eeg))
print(np.array(raw.ch_names)[np.setdiff1d(all_eeg, good_eeg)])

### When to look for bad channels

You can start looking for bad channels during the experiment session when the
data is being acquired. If you notice any flat or excessively noisy channels,
you can note them in your experiment log or protocol sheet. If your system
computes online averages, these can be a good way to spot bad channels as
well. After the data has been collected, you can do a more thorough check for
bad channels by browsing the raw data using :meth:`mne.io.Raw.plot`, without
any projectors or ICA applied. Finally, you can compute offline averages
(again with projectors, ICA, and EEG referencing disabled) to look for
channels with unusual properties. Here's an example of ERP/F plots where the
bad channels were not properly marked:

In [ ]:
raw2 = raw.copy()
raw2.info["bads"] = []
events = mne.find_events(raw2, stim_channel="STI 014")
epochs = mne.Epochs(raw2, events=events)["2"].average().plot()

The bad EEG channel is not so obvious, but the bad gradiometer is easy to
see.

Remember, marking bad channels should be done as early as possible in the
analysis pipeline. When bad channels are marked in a :class:`~mne.io.Raw`
object, the markings will be automatically transferred through the chain of
derived object types: including :class:`~mne.Epochs` and :class:`~mne.Evoked`
objects, but also :class:`noise covariance <mne.Covariance>` objects,
:class:`forward solution computations <mne.Forward>`, :class:`inverse
operators <mne.minimum_norm.InverseOperator>`, etc. If you don't notice the
badness until later stages of your analysis pipeline, you'll probably need to
go back and re-run the pipeline, so it's a good investment of time to
carefully explore the data for bad channels early on.


### Why mark bad channels at all?

Many analysis computations can be strongly affected by the presence of bad
channels. For example, a malfunctioning channel with completely flat signal
will have zero channel variance, which will cause noise estimates to be
unrealistically low. This low noise estimate will lead to a strong channel
weight in the estimate of cortical current, and because the channel is flat,
the magnitude of cortical current estimates will shrink dramatically.

Conversely, very noisy channels can also cause problems. For example, they
can lead to too many epochs being discarded based on signal amplitude
rejection thresholds, which in turn can lead to less robust estimation of the
noise covariance across sensors. Noisy channels can also interfere with
:term:`SSP` computations, because the projectors will be
spatially biased in the direction of the noisy channel, which can cause
adjacent good channels to be suppressed. ICA is corrupted by noisy channels
for similar reasons. On the other hand, when performing machine learning
analyses, bad channels may have limited, if any impact (i.e., bad channels
will be uninformative and therefore ignored / deweighted by the algorithm).


## Interpolating bad channels

In some cases simply excluding bad channels is sufficient (for example, if
you plan only to analyze a specific sensor ROI, and the bad channel is
outside that ROI). However, in cross-subject analyses it is often helpful to
maintain the same data dimensionality for all subjects, and there is no
guarantee that the same channels will be bad for all subjects. It is possible
in such cases to remove each channel that is bad for even a single subject,
but that can lead to a dramatic drop in data rank (and ends up discarding a
fair amount of clean data in the process). In such cases it is desirable to
reconstruct bad channels by interpolating its signal based on the signals of
the good sensors around them.


### How interpolation works

Interpolation of EEG channels in MNE-Python is done using the spherical
spline method :footcite:`PerrinEtAl1989`, which projects the sensor
locations onto a unit sphere
and interpolates the signal at the bad sensor locations based on the signals
at the good locations. Mathematical details are presented in
`channel-interpolation`. Interpolation of MEG channels uses the field
mapping algorithms used in computing the `forward solution
<tut-forward>`.


### Interpolation in MNE-Python

Interpolating bad channels in :class:`~mne.io.Raw` objects is done with the
:meth:`~mne.io.Raw.interpolate_bads` method, which automatically applies the
correct method (spherical splines or field interpolation) to EEG and MEG
channels, respectively (there is a corresponding method
:meth:`mne.Epochs.interpolate_bads` that works for :class:`~mne.Epochs`
objects). To illustrate how it works, we'll start by cropping the raw object
to just three seconds for easier plotting:

In [ ]:
raw.crop(tmin=0, tmax=3).load_data()

By default, :meth:`~mne.io.Raw.interpolate_bads` will clear out
``raw.info['bads']`` after interpolation, so that the interpolated channels
are no longer excluded from subsequent computations. Here, for illustration
purposes, we'll prevent that by specifying ``reset_bads=False`` so that when
we plot the data before and after interpolation, the affected channels will
still plot in red:

In [ ]:
eeg_data = raw.copy().pick(picks="eeg")
eeg_data_interp = eeg_data.copy().interpolate_bads(reset_bads=False)

for title, data in zip(["orig.", "interp."], [eeg_data, eeg_data_interp]):
    with mne.viz.use_browser_backend("matplotlib"):
        fig = data.plot(butterfly=True, color="#00000022", bad_color="r")
    fig.subplots_adjust(top=0.9)
    fig.suptitle(title, size="xx-large", weight="bold")

Note that the method :meth:`~mne.io.Raw.pick` default
arguments includes ``exclude=()`` which ensures that bad
channels are not
automatically dropped from the selection. Here is the corresponding example
with the interpolated gradiometer channel; since there are more channels
we'll use a more transparent gray color this time:

In [ ]:
grad_data = raw.copy().pick(picks="grad")
grad_data_interp = grad_data.copy().interpolate_bads(reset_bads=False)

for data in (grad_data, grad_data_interp):
    data.plot(butterfly=True, color="#00000009", bad_color="r")

## 3. Rejecting bad data spans and breaks

This tutorial covers:

- manual marking of bad spans of data,
- automated rejection of data spans based on signal amplitude, and
- automated detection of breaks during an experiment.

We begin as always by importing the necessary Python modules and loading some
`example data <sample-dataset>`; to save memory we'll use a pre-filtered
and downsampled version of the example data, and we'll also load an events
array to use when converting the continuous data to epochs:

In [ ]:
import os

import numpy as np

import mne

sample_data_folder = mne.datasets.sample.data_path()
sample_data_raw_file = os.path.join(
    sample_data_folder, "MEG", "sample", "sample_audvis_filt-0-40_raw.fif"
)
raw = mne.io.read_raw_fif(sample_data_raw_file, verbose=False)
events_file = os.path.join(
    sample_data_folder, "MEG", "sample", "sample_audvis_filt-0-40_raw-eve.fif"
)
events = mne.read_events(events_file)

## Annotating bad spans of data

The tutorial `tut-events-vs-annotations` describes how
:class:`~mne.Annotations` can be read from embedded events in the raw
recording file, and `tut-annotate-raw` describes in detail how to
interactively annotate a :class:`~mne.io.Raw` data object. Here, we focus on
best practices for annotating *bad* data spans so that they will be excluded
from your analysis pipeline.


### The ``reject_by_annotation`` parameter

In the interactive ``raw.plot()`` window, the annotation controls can be
opened by pressing :kbd:`a`. Here, new annotation labels can be created or
existing annotation labels can be selected for use.

In [ ]:
fig = raw.plot()
fig.fake_keypress("a")  # Simulates user pressing 'a' on the keyboard.

You can see that you need to add a description first to start with
marking spans (Push the button "Add Description" and enter the description).
You can use any description you like, but annotations marking spans that
should be excluded from the analysis pipeline should all begin with "BAD" or
"bad" (e.g., "bad_cough", "bad-eyes-closed", "bad door slamming", etc). When
this practice is followed, many processing steps in MNE-Python will
automatically exclude the "bad"-labelled spans of data; this behavior is
controlled by a parameter ``reject_by_annotation`` that can be found in many
MNE-Python functions or class constructors, including:

- creation of epoched data from continuous data (:class:`mne.Epochs`)
- many methods of the independent components analysis class
  (:class:`mne.preprocessing.ICA`)
- functions for finding heartbeat and blink artifacts
  (:func:`~mne.preprocessing.find_ecg_events`,
  :func:`~mne.preprocessing.find_eog_events`)
- covariance computations (:func:`mne.compute_raw_covariance`)
- power spectral density computation (:meth:`mne.io.Raw.compute_psd`)

For example, when creating epochs from continuous data, if
``reject_by_annotation=True`` the :class:`~mne.Epochs` constructor will drop
any epoch that partially or fully overlaps with an annotated span that begins
with "bad".


### Generating annotations programmatically

The `tut-artifact-overview` tutorial introduced the artifact detection
functions :func:`~mne.preprocessing.find_eog_events` and
:func:`~mne.preprocessing.find_ecg_events` (although that tutorial mostly
relied on their higher-level wrappers
:func:`~mne.preprocessing.create_eog_epochs` and
:func:`~mne.preprocessing.create_ecg_epochs`). Here, for demonstration
purposes, we make use of the lower-level artifact detection function to get
an events array telling us where the blinks are, then automatically add
"bad_blink" annotations around them (this is not necessary when using
:func:`~mne.preprocessing.create_eog_epochs`, it is done here just to show
how annotations are added non-interactively). We'll start the annotations
250 ms before the blink and end them 250 ms after it:

In [ ]:
eog_events = mne.preprocessing.find_eog_events(raw)
onsets = eog_events[:, 0] / raw.info["sfreq"] - 0.25
durations = [0.5] * len(eog_events)
descriptions = ["bad blink"] * len(eog_events)
blink_annot = mne.Annotations(
    onsets, durations, descriptions, orig_time=raw.info["meas_date"]
)
raw.set_annotations(blink_annot)

Now we can confirm that the annotations are centered on the EOG events. Since
blinks are usually easiest to see in the EEG channels, we'll only plot EEG
here:

In [ ]:
eeg_picks = mne.pick_types(raw.info, meg=False, eeg=True)
raw.plot(events=eog_events, order=eeg_picks)

See the section `tut-section-programmatic-annotations` for more details
on creating annotations programmatically.

### Detecting and annotating breaks
Another useful function, albeit not related to artifact detection *per se*,
is `mne.preprocessing.annotate_break`: It will generate annotations for
segments of the data where no existing annotations (or, alternatively:
events) can be found. It can therefore be used to automatically detect and
mark breaks, e.g. between experimental blocks, when recording continued.

For the sake of this example, let's assume an experiment consisting of two
blocks, the first one stretching from 30 to 90, and the second from 120 to
180 seconds. We'll mark these blocks by annotations, and then use
`mne.preprocessing.annotate_break` to detect and annotate any breaks.

<div class="alert alert-info"><h4>Note</h4><p>We need to take ``raw.first_time`` into account, otherwise the
          onsets will be incorrect!</p></div>

In [ ]:
onsets = [raw.first_time + 30, raw.first_time + 180]
durations = [60, 60]
descriptions = ["block_1", "block_2"]

block_annots = mne.Annotations(
    onset=onsets,
    duration=durations,
    description=descriptions,
    orig_time=raw.info["meas_date"],
)
raw.set_annotations(raw.annotations + block_annots)  # add to existing
raw.plot()

Now detect break periods. We can control how far the break annotations shall
expand toward both ends of each break.

In [ ]:
break_annots = mne.preprocessing.annotate_break(
    raw=raw,
    min_break_duration=20,  # consider segments of at least 20 s duration
    t_start_after_previous=5,  # start annotation 5 s after end of previous one
    t_stop_before_next=2,  # stop annotation 2 s before beginning of next one
)

raw.set_annotations(raw.annotations + break_annots)  # add to existing
raw.plot()

You can see that 3 segments have been annotated as ``BAD_break``:

- the first one starting with the beginning of the recording and ending 2
  seconds before the beginning of block 1 (due to ``t_stop_before_next=2``),
- the second one starting 5 seconds after block 1 has ended, and ending 2
  seconds before the beginning of block 2 (``t_start_after_previous=5``,
  ``t_stop_before_next=2``),
- and the last one starting 5 seconds after block 2 has ended
  (``t_start_after_previous=5``) and continuing until the end of the
  recording.

You can also see that only the ``block_1`` and ``block_2`` annotations
were considered in the detection of the break periods – the EOG annotations
were simply ignored. This is because, by default,
`~mne.preprocessing.annotate_break` ignores all annotations starting with
``'bad'``. You can control this behavior via the ``ignore`` parameter.

It is also possible to perform break period detection based on an array
of events: simply pass the array via the ``events`` parameter. Existing
annotations in the raw data will be ignored in this case:

In [ ]:
# only keep some button press events (code 32) for this demonstration
events_subset = events[events[:, -1] == 32]
# drop the first and last few events
events_subset = events_subset[3:-3]

break_annots = mne.preprocessing.annotate_break(
    raw=raw,
    events=events_subset,  # passing events will ignore existing annotations
    min_break_duration=25,  # pick a longer break duration this time
)

# replace existing annotations (otherwise it becomes difficult to see any
# effects in the plot!)
raw.set_annotations(break_annots)
raw.plot(events=events_subset)

## Rejecting Epochs based on peak-to-peak channel amplitude

Besides "bad" annotations, the :class:`mne.Epochs` class constructor has
another means of rejecting epochs, based on signal amplitude thresholds for
each channel type. In the `overview tutorial
<tut-section-overview-epoching>` we saw an example of this: setting maximum
acceptable peak-to-peak amplitudes for each channel type in an epoch, using
the ``reject`` parameter. There is also a related parameter, ``flat``, that
can be used to set *minimum* acceptable peak-to-peak amplitudes for each
channel type in an epoch:

In [ ]:
reject_criteria = dict(
    mag=3000e-15,  # 3000 fT
    grad=3000e-13,  # 3000 fT/cm
    eeg=100e-6,  # 100 µV
    eog=200e-6,
)  # 200 µV

flat_criteria = dict(mag=1e-15, grad=1e-13, eeg=1e-6)  # 1 fT  # 1 fT/cm  # 1 µV

The values that are appropriate are dataset- and hardware-dependent, so some
trial-and-error may be necessary to find the correct balance between data
quality and loss of power due to too many dropped epochs. Here, we've set the
rejection criteria to be fairly stringent, for illustration purposes.

Two additional parameters, ``reject_tmin`` and ``reject_tmax``, are used to
set the temporal window in which to calculate peak-to-peak amplitude for the
purposes of epoch rejection. These default to the same ``tmin`` and ``tmax``
of the entire epoch. As one example, if you wanted to only apply the
rejection thresholds to the portion of the epoch that occurs *before* the
event marker around which the epoch is created, you could set
``reject_tmax=0``. A summary of the causes of rejected epochs can be
generated with the :meth:`~mne.Epochs.plot_drop_log` method:

In [ ]:
raw.set_annotations(blink_annot)  # restore the EOG annotations
epochs = mne.Epochs(
    raw,
    events,
    tmin=-0.2,
    tmax=0.5,
    reject_tmax=0,
    reject=reject_criteria,
    flat=flat_criteria,
    reject_by_annotation=False,
    preload=True,
)
epochs.plot_drop_log()

Notice that we've passed ``reject_by_annotation=False`` above, in order to
isolate the effects of the rejection thresholds. If we re-run the epoching
with ``reject_by_annotation=True`` (the default) we see that the rejections
due to EEG and EOG channels have disappeared (suggesting that those channel
fluctuations were probably blink-related, and were subsumed by rejections
based on the "bad blink" label).

In [ ]:
epochs = mne.Epochs(
    raw,
    events,
    tmin=-0.2,
    tmax=0.5,
    reject_tmax=0,
    reject=reject_criteria,
    flat=flat_criteria,
    preload=True,
)
epochs.plot_drop_log()

More importantly, note that *many* more epochs are rejected (~12.2% instead
of ~2.5%) when rejecting based on the blink labels, underscoring why it is
usually desirable to repair artifacts rather than exclude them.

The :meth:`~mne.Epochs.plot_drop_log` method is a visualization of an
:class:`~mne.Epochs` attribute, namely ``epochs.drop_log``, which stores
empty lists for retained epochs and lists of strings for dropped epochs, with
the strings indicating the reason(s) why the epoch was dropped. For example:

In [ ]:
print(epochs.drop_log)

Finally, it should be noted that "dropped" epochs are not necessarily deleted
from the :class:`~mne.Epochs` object right away. Above, we forced the
dropping to happen when we created the :class:`~mne.Epochs` object by using
the ``preload=True`` parameter. If we had not done that, the
:class:`~mne.Epochs` object would have been `memory-mapped`_ (not loaded into
RAM), in which case the criteria for dropping epochs are stored, and the
actual dropping happens when the :class:`~mne.Epochs` data are finally loaded
and used. There are several ways this can get triggered, such as:

- explicitly loading the data into RAM with the :meth:`~mne.Epochs.load_data`
  method
- plotting the data (:meth:`~mne.Epochs.plot`,
  :meth:`~mne.Epochs.plot_image`, etc)
- using the :meth:`~mne.Epochs.average` method to create an
  :class:`~mne.Evoked` object

You can also trigger dropping with the :meth:`~mne.Epochs.drop_bad` method;
if ``reject`` and/or ``flat`` criteria have already been provided to the
epochs constructor, :meth:`~mne.Epochs.drop_bad` can be used without
arguments to simply delete the epochs already marked for removal (if the
epochs have already been dropped, nothing further will happen):

In [ ]:
epochs.drop_bad()

Alternatively, if rejection thresholds were not originally given to the
:class:`~mne.Epochs` constructor, they can be passed to
:meth:`~mne.Epochs.drop_bad` later instead; this can also be a way of
imposing progressively more stringent rejection criteria:

In [ ]:
stronger_reject_criteria = dict(
    mag=2000e-15,  # 2000 fT
    grad=2000e-13,  # 2000 fT/cm
    eeg=100e-6,  # 100 µV
    eog=100e-6,
)  # 100 µV

epochs.drop_bad(reject=stronger_reject_criteria)
print(epochs.drop_log)

## Rejecting Epochs using callables (functions)

Sometimes it is useful to reject epochs based criteria other than
peak-to-peak amplitudes. For example, we might want to reject epochs
based on the maximum or minimum amplitude of a channel.
In this case, the `mne.Epochs.drop_bad` function also accepts
callables (functions) in the ``reject`` and ``flat`` parameters. This
allows us to define functions to reject epochs based on our desired criteria.

Let's begin by generating Epoch data with large artifacts in one eeg channel
in order to demonstrate the versatility of this approach.

In [ ]:
raw.crop(0, 5)
raw.del_proj()
chans = raw.info["ch_names"][-5:-1]
raw.pick(chans)
data = raw.get_data()

new_data = data
new_data[0, 180:200] *= 1e3
new_data[0, 460:580] += 1e-3
edit_raw = mne.io.RawArray(new_data, raw.info)

# Create fixed length epochs of 1 second
events = mne.make_fixed_length_events(edit_raw, id=1, duration=1.0, start=0)
epochs = mne.Epochs(edit_raw, events, tmin=0, tmax=1, baseline=None)
epochs.plot(scalings=dict(eeg=50e-5))

As you can see, we have two large artifacts in the first channel. One large
spike in amplitude and one large increase in amplitude.

In [ ]:
# Let's try to reject the epoch containing the spike in amplitude based on the
# maximum amplitude of the first channel. Please note that the callable in
# ``reject`` must return a (good, reason) tuple. Where the good must be bool
# and reason must be a str, list, or tuple where each entry is a str.

epochs = mne.Epochs(
    edit_raw,
    events,
    tmin=0,
    tmax=1,
    baseline=None,
    preload=True,
)

epochs.drop_bad(
    reject=dict(eeg=lambda x: ((np.max(x, axis=1) > 1e-2).any(), "max amp"))
)
epochs.plot(scalings=dict(eeg=50e-5))

Here, the epoch containing the spike in amplitude was rejected for having a
maximum amplitude greater than 1e-2 Volts. Notice the use of the ``any()``
function to check if any of the channels exceeded the threshold. We could
have also used the ``all()`` function to check if all channels exceeded the
threshold.

In [ ]:
# Next, let's try to reject the epoch containing the increase in amplitude
# using the median.

epochs = mne.Epochs(
    edit_raw,
    events,
    tmin=0,
    tmax=1,
    baseline=None,
    preload=True,
)

epochs.drop_bad(
    reject=dict(eeg=lambda x: ((np.median(x, axis=1) > 1e-4).any(), "median amp"))
)
epochs.plot(scalings=dict(eeg=50e-5))

Finally, let's try to reject both epochs using a combination of the maximum
and median. We'll define a custom function and use boolean operators to
combine the two criteria.

In [ ]:
def reject_criteria(x):
    max_condition = np.max(x, axis=1) > 1e-2
    median_condition = np.median(x, axis=1) > 1e-4
    return ((max_condition.any() or median_condition.any()), ["max amp", "median amp"])


epochs = mne.Epochs(
    edit_raw,
    events,
    tmin=0,
    tmax=1,
    baseline=None,
    preload=True,
)

epochs.drop_bad(reject=dict(eeg=reject_criteria))
epochs.plot(events=True)

Note that a complementary Python module, the `autoreject package`_, uses
machine learning to find optimal rejection criteria, and is designed to
integrate smoothly with MNE-Python workflows. This can be a considerable
time-saver when working with heterogeneous datasets.

## 4. Filtering and resampling data

This tutorial covers filtering and resampling, and gives examples of how
filtering can be used for artifact repair.

We begin as always by importing the necessary Python modules and loading some
`example data <sample-dataset>`. We'll also crop the data to 60 seconds
(to save memory on the documentation server):

In [ ]:
import os

import matplotlib.pyplot as plt
import numpy as np

import mne

sample_data_folder = mne.datasets.sample.data_path()
sample_data_raw_file = os.path.join(
    sample_data_folder, "MEG", "sample", "sample_audvis_raw.fif"
)
raw = mne.io.read_raw_fif(sample_data_raw_file)
# use just 60 seconds of data and mag channels, to save memory
raw.crop(0, 60).pick(picks=["mag", "stim"]).load_data()

## Background on filtering

A filter removes or attenuates parts of a signal. Usually, filters act on
specific *frequency ranges* of a signal — for example, suppressing all
frequency components above or below a certain cutoff value. There are *many*
ways of designing digital filters; see `disc-filtering` for a longer
discussion of the various approaches to filtering physiological signals in
MNE-Python.


## Repairing artifacts by filtering

Artifacts that are restricted to a narrow frequency range can sometimes
be repaired by filtering the data. Two examples of frequency-restricted
artifacts are slow drifts and power line noise. Here we illustrate how each
of these can be repaired by filtering.


### Slow drifts

Low-frequency drifts in raw data can usually be spotted by plotting a fairly
long span of data with the :meth:`~mne.io.Raw.plot` method, though it is
helpful to disable channel-wise DC shift correction to make slow drifts
more readily visible. Here we plot 60 seconds, showing all the magnetometer
channels:

In [ ]:
raw.plot(duration=60, proj=False, n_channels=len(raw.ch_names), remove_dc=False)

A half-period of this slow drift appears to last around 10 seconds, so a full
period would be 20 seconds, i.e., $\frac{1}{20} \mathrm{Hz}$. To be
sure those components are excluded, we want our highpass to be *higher* than
that, so let's try $\frac{1}{10} \mathrm{Hz}$ and $\frac{1}{5}
\mathrm{Hz}$ filters to see which works best:

In [ ]:
for cutoff in (0.1, 0.2):
    raw_highpass = raw.copy().filter(l_freq=cutoff, h_freq=None)
    with mne.viz.use_browser_backend("matplotlib"):
        fig = raw_highpass.plot(
            duration=60, proj=False, n_channels=len(raw.ch_names), remove_dc=False
        )
    fig.subplots_adjust(top=0.9)
    fig.suptitle(f"High-pass filtered at {cutoff} Hz", size="xx-large", weight="bold")

Looks like 0.1 Hz was not quite high enough to fully remove the slow drifts.
Notice that the text output summarizes the relevant characteristics of the
filter that was created. If you want to visualize the filter, you can pass
the same arguments used in the call to :meth:`raw.filter()
<mne.io.Raw.filter>` above to the function :func:`mne.filter.create_filter`
to get the filter parameters, and then pass the filter parameters to
:func:`mne.viz.plot_filter`. :func:`~mne.filter.create_filter` also requires
parameters ``data`` (a :class:`NumPy array <numpy.ndarray>`) and ``sfreq``
(the sampling frequency of the data), so we'll extract those from our
:class:`~mne.io.Raw` object:

In [ ]:
filter_params = mne.filter.create_filter(
    raw.get_data(), raw.info["sfreq"], l_freq=0.2, h_freq=None
)

Notice that the output is the same as when we applied this filter to the data
using :meth:`raw.filter() <mne.io.Raw.filter>`. You can now pass the filter
parameters (and the sampling frequency) to :func:`~mne.viz.plot_filter` to
plot the filter:

In [ ]:
mne.viz.plot_filter(filter_params, raw.info["sfreq"], flim=(0.01, 5))

### Power line noise

Power line noise is an environmental artifact that manifests as persistent
oscillations centered around the `AC power line frequency`_. Power line
artifacts are easiest to see on plots of the spectrum, so we'll use
:meth:`~mne.io.Raw.compute_psd` to get a
:class:`~mne.time_frequency.Spectrum` object, and use its
:meth:`~mne.time_frequency.Spectrum.plot` method to illustrate. We'll also
write a little function that adds arrows to the spectrum plot to highlight
the artifacts:

In [ ]:
def add_arrows(axes):
    """Add some arrows at 60 Hz and its harmonics."""
    for ax in axes:
        freqs = ax.lines[-1].get_xdata()
        psds = ax.lines[-1].get_ydata()
        for freq in (60, 120, 180, 240):
            idx = np.searchsorted(freqs, freq)
            # get ymax of a small region around the freq. of interest
            y = psds[(idx - 4) : (idx + 5)].max()
            ax.arrow(
                x=freqs[idx],
                y=y + 18,
                dx=0,
                dy=-12,
                color="red",
                width=0.1,
                head_width=3,
                length_includes_head=True,
            )


fig = raw.compute_psd(fmax=250).plot(
    average=True, amplitude=False, picks="data", exclude="bads"
)
add_arrows(fig.axes[:2])

It should be evident that MEG channels are more susceptible to this kind of
interference than EEG that is recorded in the magnetically shielded room.
Removing power-line noise can be done with a notch filter,
applied directly to the :class:`~mne.io.Raw` object, specifying an array of
frequencies to be attenuated. Since the EEG channels are relatively
unaffected by the power line noise, we'll also specify a ``picks`` argument
so that only the magnetometers and gradiometers get filtered:

In [ ]:
meg_picks = mne.pick_types(raw.info, meg=True)
freqs = (60, 120, 180, 240)
raw_notch = raw.copy().notch_filter(freqs=freqs, picks=meg_picks)
for title, data in zip(["Un", "Notch "], [raw, raw_notch]):
    fig = data.compute_psd(fmax=250).plot(
        average=True, amplitude=False, picks="data", exclude="bads"
    )
    fig.suptitle(f"{title}filtered", size="xx-large", weight="bold")
    add_arrows(fig.axes[:2])

`~mne.io.Raw.notch_filter` also has parameters to control the notch
width, transition bandwidth and other aspects of the filter. See the
docstring for details.

It's also possible to try to use a spectrum fitting routine to notch filter.
In principle it can automatically detect the frequencies to notch, but our
implementation generally does not do so reliably, so we specify the
frequencies to remove instead, and it does a good job of removing the
line noise at those frequencies:

In [ ]:
raw_notch_fit = raw.copy().notch_filter(
    freqs=freqs, picks=meg_picks, method="spectrum_fit", filter_length="10s"
)
for title, data in zip(["Un", "spectrum_fit "], [raw, raw_notch_fit]):
    fig = data.compute_psd(fmax=250).plot(
        average=True, amplitude=False, picks="data", exclude="bads"
    )
    fig.suptitle(f"{title}filtered", size="xx-large", weight="bold")
    add_arrows(fig.axes[:2])

## Resampling

EEG and MEG recordings are notable for their high temporal precision, and are
often recorded with sampling rates around 1000 Hz or higher. This is good
when precise timing of events is important to the experimental design or
analysis plan, but also consumes more memory and computational resources when
processing the data. In cases where high-frequency components of the signal
are not of interest and precise timing is not needed (e.g., computing EOG or
ECG projectors on a long recording), downsampling the signal can be a useful
time-saver.

In MNE-Python, the resampling methods (:meth:`raw.resample()
<mne.io.Raw.resample>`, :meth:`epochs.resample() <mne.Epochs.resample>` and
:meth:`evoked.resample() <mne.Evoked.resample>`) apply a low-pass filter to
the signal to avoid `aliasing`_, so you don't need to explicitly filter it
yourself first. This built-in filtering that happens when using
:meth:`raw.resample() <mne.io.Raw.resample>`, :meth:`epochs.resample()
<mne.Epochs.resample>`, or :meth:`evoked.resample() <mne.Evoked.resample>` is
a brick-wall filter applied in the frequency domain at the `Nyquist
frequency`_ of the desired new sampling rate. This can be clearly seen in the
PSD plot, where a dashed vertical line indicates the filter cutoff; the
original data had an existing lowpass at around 172 Hz (see
``raw.info['lowpass']``), and the data resampled from ~600 Hz to 200 Hz gets
automatically lowpass filtered at 100 Hz (the `Nyquist frequency`_ for a
target rate of 200 Hz):

In [ ]:
raw_downsampled = raw.copy().resample(sfreq=200)
# choose n_fft for Welch PSD to make frequency axes similar resolution
n_ffts = [4096, int(round(4096 * 200 / raw.info["sfreq"]))]
fig, axes = plt.subplots(2, 1, sharey=True, layout="constrained", figsize=(10, 6))
for ax, data, title, n_fft in zip(
    axes, [raw, raw_downsampled], ["Original", "Downsampled"], n_ffts
):
    fig = data.compute_psd(n_fft=n_fft).plot(
        average=True, amplitude=False, picks="data", exclude="bads", axes=ax
    )
    ax.set(title=title, xlim=(0, 300))

By default, MNE-Python resamples using ``method="fft"``, which performs FFT-based
resampling via :func:`scipy.signal.resample`. While efficient and good for most
biological signals, it has two main potential drawbacks:

1. It assumes periodicity of the signal. We try to overcome this with appropriate
   signal padding, but some signal leakage may still occur.
2. It treats the entire signal as a single block. This means that in general effects
   are not guaranteed to be localized in time, though in practice they often are.

Alternatively, resampling can be performed using ``method="polyphase"`` instead.
This uses :func:`scipy.signal.resample_poly` under the hood, which in turn utilizes
a three-step process to resample signals (see :func:`scipy.signal.upfirdn` for
details). This process guarantees that each resampled output value is only affected by
input values within a limited range. In other words, output values are guaranteed to
be a result of a specific set of input values.

In general, using ``method="polyphase"`` can also be faster than ``method="fft"`` in
cases where the desired sampling rate is an integer factor different from the input
sampling rate. For example:

In [ ]:
n_ffts = [4096, 2048]  # factor of 2 smaller n_fft
raw_downsampled_poly = raw.copy().resample(
    sfreq=raw.info["sfreq"] / 2.0,
    method="polyphase",
    verbose=True,
)
fig, axes = plt.subplots(2, 1, sharey=True, layout="constrained", figsize=(10, 6))
for ax, data, title, n_fft in zip(
    axes, [raw, raw_downsampled_poly], ["Original", "Downsampled (polyphase)"], n_ffts
):
    data.compute_psd(n_fft=n_fft).plot(
        average=True, amplitude=False, picks="data", exclude="bads", axes=ax
    )
    ax.set(title=title, xlim=(0, 300))

Because resampling involves filtering, there are some pitfalls to resampling
at different points in the analysis stream:

- Performing resampling on :class:`~mne.io.Raw` data (*before* epoching) will
  negatively affect the temporal precision of Event arrays, by causing
  `jitter`_ in the event timing. This reduced temporal precision will
  propagate to subsequent epoching operations.

- Performing resampling *after* epoching can introduce edge artifacts *on
  every epoch*, whereas filtering the :class:`~mne.io.Raw` object will only
  introduce artifacts at the start and end of the recording (which is often
  far enough from the first and last epochs to have no affect on the
  analysis).

The following section suggests best practices to mitigate both of these
issues.


### Best practices

To avoid the reduction in temporal precision of events that comes with
resampling a :class:`~mne.io.Raw` object, and also avoid the edge artifacts
that come with filtering an :class:`~mne.Epochs` or :class:`~mne.Evoked`
object, the best practice is to:

1. low-pass filter the :class:`~mne.io.Raw` data at or below
   $\frac{1}{3}$ of the desired sample rate, then

2. decimate the data after epoching, by either passing the ``decim``
   parameter to the :class:`~mne.Epochs` constructor, or using the
   :meth:`~mne.Epochs.decimate` method after the :class:`~mne.Epochs` have
   been created.

<div class="alert alert-danger"><h4>Warning</h4><p>The recommendation for setting the low-pass corner frequency at
   $\frac{1}{3}$ of the desired sample rate is a fairly safe rule of
   thumb based on the default settings in :meth:`raw.filter()
   <mne.io.Raw.filter>` (which are different from the filter settings used
   inside the :meth:`raw.resample() <mne.io.Raw.resample>` method). If you
   use a customized lowpass filter (specifically, if your transition
   bandwidth is wider than 0.5× the lowpass cutoff), downsampling to 3× the
   lowpass cutoff may still not be enough to avoid `aliasing`_, and
   MNE-Python will not warn you about it (because the :class:`raw.info
   <mne.Info>` object only keeps track of the lowpass cutoff, not the
   transition bandwidth). Conversely, if you use a steeper filter, the
   warning may be too sensitive. If you are unsure, plot the PSD of your
   filtered data *before decimating* and ensure that there is no content in
   the frequencies above the `Nyquist frequency`_ of the sample rate you'll
   end up with *after* decimation.</p></div>

Note that this method of manually filtering and decimating is exact only when
the original sampling frequency is an integer multiple of the desired new
sampling frequency. Since the sampling frequency of our example data is
600.614990234375 Hz, ending up with a specific sampling frequency like (say)
90 Hz will not be possible:

In [ ]:
current_sfreq = raw.info["sfreq"]
desired_sfreq = 90  # Hz
decim = np.round(current_sfreq / desired_sfreq).astype(int)
obtained_sfreq = current_sfreq / decim
lowpass_freq = obtained_sfreq / 3.0

raw_filtered = raw.copy().filter(l_freq=None, h_freq=lowpass_freq)
events = mne.find_events(raw_filtered)
epochs = mne.Epochs(raw_filtered, events, decim=decim)

print(
    "desired sampling frequency was {} Hz; decim factor of {} yielded an "
    "actual sampling frequency of {} Hz.".format(
        desired_sfreq, decim, epochs.info["sfreq"]
    )
)

If for some reason you cannot follow the above-recommended best practices,
you should at the very least either:

1. resample the data *after* epoching, and make your epochs long enough that
   edge effects from the filtering do not affect the temporal span of the
   epoch that you hope to analyze / interpret; or

2. perform resampling on the :class:`~mne.io.Raw` object and its
   corresponding Events array *simultaneously* so that they stay more or less
   in synch. This can be done by passing the Events array as the
   ``events`` parameter to :meth:`raw.resample() <mne.io.Raw.resample>`.